In [5]:
import cassiopeia as cass
from cassiopeia.core.league import ChallengerLeague, GrandmasterLeague, MasterLeague
from cassiopeia.datastores.riotapi.common import APIError
from dotenv import load_dotenv
import os
from itertools import islice
import json
import time
import pickle
import io
import gzip
load_dotenv() 


True

### First Step: Fetching Data From API (CAN BE BUGGY WITH CASSOPEIA LIBRARY, WHICH IS OUTDATED)

In [14]:
curr_dir = os.getcwd()
data_dir = os.path.join(curr_dir, "..", "..", "data")

In [8]:
cass.set_riot_api_key(os.getenv("RIOT_API_KEY")) # make sure you have a .env file with this key in it


In [9]:
# the package's lazy loader is bugged, so this method is needed to extract raw data; warning, this can be buggy and my not work as the package is outdated
def extract_raw_data(obj):
    data = obj._data 
    raw_data = data[next(iter(data))] 
    return raw_data

In [10]:
# gives you all the puuids in a league (challenger, grandmaster, master)
def get_all_league_entries(league):
    uuids = []

    for entry in league:
        raw_data = extract_raw_data(entry)
        uuids.append(raw_data.puuid)
    return uuids

In [11]:
# get all the players in the top ranks
challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives, region="NA")
grandmaster_league = cass.get_grandmaster_league(queue=cass.Queue.ranked_solo_fives, region="NA")
master_league = cass.get_master_league(queue=cass.Queue.ranked_solo_fives, region="NA")


The code below which is commented out essentially loops through each player in the TOP leaderboards, and then loops through each of their previous games and adds it to the list. However this runs an API call for each match, so it will take a while to run. So I saved it as a pickle file that can be loaded in the cell two below. 

In [ ]:
# takes around 2 hours to run this


# for NA challenger matches

# all_chall_players_puuid =  get_all_league_entries(challenger_league)
# all_chall_matches = []
# with open("na_chall_matches.pkl", "wb") as f:
#     pickle.dump(all_chall_matches, f)
# # for NA grandmaster matches

# all_gm_players_puuid =  get_all_league_entries(grandmaster_league)
# all_gm_matches = []
# for puuid in all_gm_players_puuid:
#     all_gm_matches.extend(cass.get_match_history(puuid=puuid, continent="AMERICAS"))

# with open("na_gm_matches.pkl", "wb") as f:
#     pickle.dump(all_gm_matches, f)

    

In [21]:
na_chall_matches = pickle.load(open(os.path.join(data_dir, "sai_pickled_matches", "na_chall_matches.pkl"), "rb"))  
na_gm_matches = pickle.load(open(os.path.join(data_dir,"sai_pickled_matches",  "na_gm_matches.pkl"), "rb"))  


In [22]:
# Verify data was loaded
print(f"Total Matches Collected:"
      f"\nNA Challenger Matches: {len(na_chall_matches)}"
      f"\nNA Grandmaster Matches: {len(na_gm_matches)}")


Total Matches Collected:
NA Challenger Matches: 242963
NA Grandmaster Matches: 557994


In [23]:
na_chall_matches.extend(na_gm_matches)
total_matches = na_chall_matches.copy()
print(f"Total Matches: {len(total_matches)}")

Total Matches: 800957


Now that we have all the matches in total_matches (lazily evaluated), we will loop in batches (Determined by BATCH_SIZE) through each match, fetching a JSON file that represents the entire match, and storing the entire batch of JSONs in a zip. The result of me running this script is stored in data/sai_match_data. (I did not process every match, it would take too long)

In [ ]:

def chunks(iterable, size):
    it = iter(iterable)
    while True:
        chunk = list(islice(it, size))
        if not chunk:
            break
        yield chunk

visited = set()
BATCH_SIZE = 2000
RETRY_DELAY = 300  


for batch_num, batch in enumerate(chunks(total_matches, BATCH_SIZE), start=1):
    print(f"\nBatch {batch_num} ({len(batch)} matches):")

    batch_data = []
    for match in batch:
        try:
            q_type = str(match.queue)
            remake = match.is_remake
            match_id = match.id
            participants = match.participants

        except KeyError:
            print(f"Skipping match: Unknown queue ID")
            continue
        except AttributeError:
            print(f"Skipping match: Missing queue info")
            continue
        except APIError as e:
            print(f"Riot API error encountered: {e}. Skipping...")
            time.sleep(RETRY_DELAY)
            continue
        except Exception as e:
            print(f"Skipping match : Unexpected error getting queue ({e})")
            time.sleep(RETRY_DELAY)
            continue
        try: 
            if (q_type == "Queue.ranked_solo_fives" and not remake):
                if match_id not in visited:
                    dict_data = json.loads(match.to_json())
                    batch_data.append(dict_data)
                    visited.add(match.id)
        except Exception as e:
            print(f"Error processing match {match.id}: {e}")
            print("Skipping this match...")
            time.sleep(RETRY_DELAY)
            continue

 
    # except Exception as e:
    #     print(f"Error in batch {batch_num}: {e}")
    #     print(f"Moving onto next batch after 2.5 minutes...")
    #     time.sleep(RETRY_DELAY)


Batch 1 (2000 matches):
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5418233073
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5418227132
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5418213545
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5418192529
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5418164436
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5418136090
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5418106839
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5418063719
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5415435203
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5410968282
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5408986667
Making call: https://americas.api.riotgames.com/lol